<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install nltk

Загрузим библиотеки

In [2]:
import pandas as pd
import re
from pymystem3 import Mystem

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

from tqdm import notebook
import numpy as np

Загрузим и посмотрим данные:

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Похоже при экспорте csv скопировался лишний столбец - удалим его.

In [5]:
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Посмотрим пропущенные значения и проверим баланс классов:

In [7]:
missing_values = df.isnull().sum()
class_balance = df['toxic'].value_counts(normalize=True)
duplicated = df.duplicated().sum()
print('Колличество пропущенных значений:\n', missing_values)
print(60*'*')
print('Баланс классов:\n',class_balance )
print(60*'*')
print('Дубликатов:', duplicated)
print(60*'*')

Колличество пропущенных значений:
 text     0
toxic    0
dtype: int64
************************************************************
Баланс классов:
 0    0.898388
1    0.101612
Name: toxic, dtype: float64
************************************************************
Дубликатов: 0
************************************************************


Пропущенных значений и дубликатов нет, это хорошо. Так же видим сильный дисбаланс классов: 90% комментариев не являются токсичными.

Создадим функцию для очистки текста и функцию для лемматизации

In [8]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = ' '.join(text.split())
    return text

In [9]:
L = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
def lemmatize_text(corpus):
    corpus_new = []
    for sentence in corpus:
        word_list = nltk.word_tokenize(sentence)
        lemm_text = ' '.join([L.lemmatize(w) for w in word_list if not w in stopwords.words('english')])
        corpus_new.append(lemm_text)
    return corpus_new

Применим функции

In [11]:
df['text'] = df['text'].apply(clear_text)

In [ ]:
df['text'] = lemmatize_text(df['text'])

1

In [ ]:
df.head(10)

,text,toxic
0,explanation edits made username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestion improvement wondered sect...,0
4,sir hero chance remember page,0
5,congratulation well use tool well talk,0
6,cocksucker piss around work,1
7,vandalism matt shirvington article reverted pl...,0
8,sorry word nonsense offensive anyway intending...,0
9,alignment subject contrary dulithgow,0


Лемматизация

## Обучение

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны